In [1]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
from __future__ import division
%load_ext autoreload
%autoreload 2
import DCMFlow_094 as dcm
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import OrderedDict
import time
import random
import operator
import random as rn
import os
from datetime import datetime
import math
import copy
import matplotlib.pyplot as plt
import os
import collections
from scipy.stats import truncnorm
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Functions to generate nested logit tree and utility expressions

In [3]:
def generate_nltree(strct=None):
    n_levels = len(strct)
    nltree_dic = {'l0':'Root', 'l1':['L1C' + str(n) for n in range(1, strct[0]+1)]}
    for level in range(1, n_levels):
        level_nests = []
        branch_cntr = 1
        for p_node in nltree_dic['l'+str(level)]:
            for j in range(1, strct[level]+1):
                j_node = p_node + '_' + 'L%dC%d'%(level+1, branch_cntr)
                level_nests.append(j_node)
                branch_cntr += 1
        nltree_dic['l'+str(level+1)] = level_nests
    return nltree_dic

In [4]:
def generate_random_choices2covariates_map(n_choices, n_covariates):
    max_combs = 2**n_covariates
    d = np.random.choice(max_combs, max_combs, replace=False)
    cov2chc_map = np.zeros(shape=[n_covariates, n_choices], dtype=int)
    colum = 0
    mn_mx_n_covs = 0
    for c in range(max_combs):
        fv = list(map(int, list(np.binary_repr(d[c], width=n_covariates))))
        c_nftrs = sum(fv)
        if c_nftrs > mn_mx_n_covs and c_nftrs <= n_covariates:
            cov2chc_map[:, colum] = fv
            colum += 1
        if colum==n_choices:
            break
    assert colum==n_choices
    return np.transpose(cov2chc_map*np.arange(1, n_covariates+1)[:, None])

def create_expression(a, pnames):
    non_zeros = []
    for i, ai in enumerate(a):
        if ai != 0:
            pname = pnames[i]
            xname = '' if pname[0]=='A' else '*X'+str(i+1)
            non_zeros.append(pname+xname)
    return '+'.join(non_zeros)

def generate_random_utilities(nltree_dict, n_covariates):
    n_levels = len(nltree_dict) - 1
    choices = nltree_dict['l'+str(n_levels)]
    n_choices = len(choices)
    beta_map = generate_random_choices2covariates_map(n_choices, n_covariates)
    alpha_map = beta_map.copy()
    alpha_map[alpha_map > 0] = 1
    uts = collections.OrderedDict()
    coefs_indices = np.unique(beta_map)
    n_coefficients = len(coefs_indices[coefs_indices>0])
    coefficient_names = ['B'+str(c) for c in range(1, n_coefficients+1)]
    const_names = ['A'+str(c) for c in range(1, alpha_map.shape[1]+1)]
    for choice in range(n_choices):
        ut_eq_alpha = create_expression(alpha_map[choice,:], const_names)
        ut_eq_beta = create_expression(beta_map[choice,:], coefficient_names)
        uts[choices[choice]] = '+'.join([ut_eq_alpha, ut_eq_beta])
    return uts

### A function to generate random covariate data

In [5]:
def generate_data(nltree_dict, n_cases, 
                  n_covariates, 
                  avail_rate_mean=0.1, 
                  avail_rate_sd=0.05,
                  covs_means_mn=0.5,
                  covs_means_mx=1.5,
                  covs_std_from_means=0.1,
                  use_choice_ids=True):
    
    n_levels = len(nltree_dict) - 1
    choices = nltree_dict['l' + str(n_levels)]
    n_choices = len(choices)
    print('n_choices = %d'%(n_choices))
    def gen_choices(n_avail_choices):
        n_avail_choices = min(n_choices, n_avail_choices)
        if use_choice_ids:
            avail_choices = list(np.sort(np.random.choice(choices, n_avail_choices, replace=False)))
        else:
            avail_choices = list(np.sort(np.random.choice(n_choices, n_avail_choices, replace=False)))
        return avail_choices
    
    loc = avail_rate_mean*n_choices
    scale = avail_rate_sd*n_choices
    n_avail_choices = truncnorm(a=2, b=n_choices, loc=loc, scale=scale).rvs(size=n_cases).astype(int)
    choice_ids_list = list(map(gen_choices, n_avail_choices))
    case_ids = []
    choice_ids = []
    for caseid in range(len(choice_ids_list)):
        sub_l = choice_ids_list[caseid]
        case_ids.extend([caseid]*len(sub_l))
        choice_ids.extend(sub_l)
    ncases_times_n_avail_choices = len(choice_ids)
    covs_means = np.random.uniform(covs_means_mn, covs_means_mx, n_covariates)
    covs = np.random.normal(loc=covs_means, scale=covs_std_from_means*covs_means, 
                            size=[ncases_times_n_avail_choices, n_covariates])
    cov_names = ['X%d'%(i+1) for i in range(n_covariates)]
    d_pd = pd.DataFrame(data=covs, columns = cov_names)
    d_pd['caseid'] = case_ids
    d_pd['choiceid'] = choice_ids
    return d_pd

### Functions to generate random parameter values and constraints

In [6]:
def generate_random_parameters(nltree_dict, n_covariates=None, theta_bins=None, 
                          theta_mn=0.65, theta_mx=0.95):
    n_levels = len(nltree_dict) - 1
    bins = theta_bins
    if bins is None:
        bnds = np.linspace(start=theta_mn, stop=theta_mx, num=n_levels)
        bins = [[bnds[i], None] 
                if i < n_levels-2 else 
                [bnds[i], bnds[i+1]] 
                for i in range(n_levels-1)][::-1]
    params_dict = {}
    logsum_level = []
    for node_name in nltree_dict['l1']:
        logsum = np.random.uniform(bins[0][0], bins[0][1])
        params_dict[node_name] = logsum
        logsum_level.append(logsum)
    logsum_levels = [copy.deepcopy(logsum_level)]
    for level in range(1, n_levels-1):
        logsum_level = []
        for i, p_node in enumerate(nltree_dict['l'+str(level)]):
            for j, c_node in enumerate(nltree_dict['l'+str(level+1)]):
                if p_node in c_node:
                    logsum = np.random.uniform(bins[level][0], logsum_levels[level-1][i])
                    params_dict[c_node] = logsum
                    logsum_level.append(logsum)
            logsum_levels.append(logsum_level)
    n_choices = len(nltree_dict['l' + str(n_levels)])
    n_covariates = n_choices if n_covariates is None else n_covariates
    for i in range(n_covariates):
        params_dict['A%d'%(i+1)] = np.round(np.random.uniform(1.0, 2.0), 2) 
        params_dict['B%d'%(i+1)] = np.round(np.random.uniform(-2.0, -0.1), 2) 
    return params_dict

def get_logsum_constraints(nltree_dict):
    n_levels = len(nltree_dict) - 1
    constraints = {}
    for level in range(1, n_levels):
        nests = nltree_dict['l%d'%(level)]
        for nest in nests:
            constraints[nest] = (0, 1.0)
    return constraints

In [7]:
n_covariates = 20
nltree_dict = generate_nltree(strct=[3,4,5,10])
#nltree_dic = generate_nltree(strct=[2,3,4])
utilities_dict = generate_random_utilities(nltree_dict, n_covariates)

In [8]:
n_cases = 80000
data = generate_data(nltree_dict, n_cases=n_cases, n_covariates=n_covariates, 
                     avail_rate_mean=0.5, avail_rate_sd=0.1,
                     covs_std_from_means=0.25)

n_choices = 600


In [9]:
true_params_dict = generate_random_parameters(nltree_dict, n_covariates=n_covariates, 
                                         theta_mn=0.6, theta_mx=0.9)

### Instantiate an NLFlow model

In [14]:
m = dcm.NLFlow(nltree_dict=nltree_dict, utilities_dict=utilities_dict)

### Simulate choices & compute loglikelihood

In [11]:
choices_and_ll = m.compute_choices_and_likelihood(data, true_params_dict)
if choices_and_ll is not None:
    simulated_choices, ll = choices_and_ll
    data_with_y = data.copy()
    data_with_y['chosen'] = simulated_choices['chosen'].copy()
    print('Loglikelihood (sum/mean) = %.1f/%.3f'%(-ll, -ll/n_cases))

Loglikelihood (sum/mean) = 298029.0/3.725


### Estimate the model's parameters using the L-BFGS-B optimizer (and compare them with true values)

In [15]:
constraints_dict = get_logsum_constraints(nltree_dict)
m.fit(data_with_y, optimizer='l-bfgs-b',
      true_params_dict=true_params_dict, 
      constraints_dict=constraints_dict)

Digesting the data...

Starting the optimization process using l-bfgs-b


### Print estimated and true parameters side by side

In [16]:
m.get_estimated_parameters(true_params_dict=true_params_dict)

ERROR: Model's parameters have not been estimated yet. Run 'fit()' first to estimate them.


### You could employ Adam to optimize using stochastic-gradient descent

In [17]:
opt = dcm.AdamOptimizer(n_steps=25000, log_every_n_epochs=1, 
                        step_size=1000, learning_rate=2e-4, 
                        patience=2000, epsilon=1e-8,
                        interior_point_penalty_term=0.0005,
                        objective='loglikelihood_sum')

In [ ]:
m.fit(data_with_y, optimizer=opt, 
      true_params_dict=true_params_dict,
      constraints_dict=constraints_dict)

Digesting the data...

Starting the optimization process using adam


Abbreviations key:
S: Step, E: Epoch, L: Loglikelihood, S/M: Sum/Mean
C: Cost (loglikehood + penalty terms for constrained 
   parameters, printed when constraints are used)
T: Time, Cf/Ls/Cn: Coefficient/Logsum/Constant
MAPE: Mean Absolute Percentage Error (printed when true
      parameters are provided)
RMSE: Root Mean Square Error (printed when true parameters
      are provided)


S/E=80/1, L(S/M)=481151.8/6.014, C(S/M)=481159.0/6.014, T=212s, Cf/Ls/Cn MAPE=102.1/30.9/99.4%, RMSE=1.134/0.214/1.420
S/E=160/2, L(S/M)=469714.7/5.871, C(S/M)=469721.8/5.872, T=423s, Cf/Ls/Cn MAPE=104.1/30.1/98.7%, RMSE=1.138/0.210/1.409
S/E=240/3, L(S/M)=459411.0/5.743, C(S/M)=459418.0/5.743, T=631s, Cf/Ls/Cn MAPE=106.1/29.2/98.0%, RMSE=1.142/0.205/1.399
S/E=320/4, L(S/M)=450108.1/5.626, C(S/M)=450115.1/5.626, T=841s, Cf/Ls/Cn MAPE=108.0/28.2/97.3%, RMSE=1.146/0.201/1.389


##### The optimization failed here, consider increasing the value of interior_point_penalty_term and/or the values of other hyper-parameters